- [x] create table
- [x] count number of rows in table
- [x] insert new data point
- [x] insert data point but failed
- [x] select data point, and count number of returns, get dict

#### Use Session Begin to do commit() automatically

In [1]:
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"

engine_config = template_engine.format(
        db_username="postgres",
        db_password="jHDSJEgp8W3rVf5",
        db_host="face-guardian-db.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="postgres")


engine = create_engine(engine_config,
    echo=True
) 

In [5]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey

from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

class User(Base):
    
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30), primary_key=True)
    fullname: Mapped[Optional[str]]
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"
    



## create table

In [6]:
import sqlalchemy
try:
    
    #checkfirst is optional, implicitly set to True so it wont contribute to error
    Base.metadata.create_all(engine, checkfirst = False) 
    
except sqlalchemy.exc.ProgrammingError as e:
    
    print("!!!Failed to create table")
    print(e)
    

2023-03-17 13:48:44,929 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 13:48:44,932 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id, name)
)


2023-03-17 13:48:44,933 INFO sqlalchemy.engine.Engine [no key 0.00150s] {}
2023-03-17 13:48:44,947 INFO sqlalchemy.engine.Engine ROLLBACK
!!!Failed to create table
(psycopg.errors.DuplicateTable) relation "user_account" already exists
[SQL: 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id, name)
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)


## insert new data point

In [7]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError


Session = sessionmaker(engine)

with Session.begin() as session:
    
    try:
        
        squidward = User(id = 0, name="squidward", fullname="Squidward Tentacles")
        krabs = User(id = 1, name="ehkrabs", fullname="Eugene H. Krabs")
        session.add(squidward)
        session.add(krabs)

        ### check the state of Session using `Session.new`
        print(session.new)

        print("Before flushing")
        print(squidward.id)
        print(krabs.id)


        ###flushing (create sql queries)
        session.flush()

        ### autogenerated id attributes
        print("After flushing")
        print(squidward.id)
        print(krabs.id)
    
    except IntegrityError as e:
        
        print("!!!Found error!!!")
        print(e)
    
    ### commit is not necessary with Session begin 
    #session.commit()
    
    ### session close is also called implicitly
    #session.close()

IdentitySet([User(id=0, name='squidward', fullname='Squidward Tentacles'), User(id=1, name='ehkrabs', fullname='Eugene H. Krabs')])
Before flushing
0
1
2023-03-17 13:48:46,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 13:48:46,296 INFO sqlalchemy.engine.Engine INSERT INTO user_account (id, name, fullname) VALUES (%(id)s::INTEGER, %(name)s::VARCHAR(30), %(fullname)s::VARCHAR)
2023-03-17 13:48:46,297 INFO sqlalchemy.engine.Engine [generated in 0.00134s] [{'id': 0, 'name': 'squidward', 'fullname': 'Squidward Tentacles'}, {'id': 1, 'name': 'ehkrabs', 'fullname': 'Eugene H. Krabs'}]
After flushing
0
1
2023-03-17 13:48:46,385 INFO sqlalchemy.engine.Engine COMMIT


## insert data point, but failed

In [8]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError


Session = sessionmaker(engine)

with Session.begin() as session:
    
    try:
        
        squidward = User(id = 3, ngme="squidward", fullname="Squidward Tentacles")
        session.add(squidward)

        ### check the state of Session using `Session.new`
        print(session.new)

        print("Before flushing")
        print(squidward.id)


        ###flushing (create sql queries)
        session.flush()

        ### autogenerated id attributes
        print("After flushing")
        print(squidward.id)
    
    except TypeError as e:
        
        print("!!!Key Error!!!")
        print(e)
    

!!!Key Error!!!
'ngme' is an invalid keyword argument for User


## select data point

In [55]:
from sqlalchemy import select

with Session.begin() as session:

    #select with filter
    users = session.execute(select(User))
    output = users.mappings().all()
    
    print("-----------")
    print(output)
    print("-----------")
    for o in output:
        print(o.get("User").name)

2023-03-17 14:04:02,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 14:04:02,075 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-03-17 14:04:02,077 INFO sqlalchemy.engine.Engine [cached since 907.8s ago] {}
-----------
[{'User': User(id=0, name='squidward', fullname='Squidward Tentacles')}, {'User': User(id=1, name='ehkrabs', fullname='Eugene H. Krabs')}]
-----------
User(id=0, name='squidward', fullname='Squidward Tentacles')
User(id=1, name='ehkrabs', fullname='Eugene H. Krabs')
2023-03-17 14:04:02,091 INFO sqlalchemy.engine.Engine COMMIT


### select key but not found 

In [56]:
with Session.begin() as session:

    #select with filter
    ehkrabsclone = session.execute(select(User).filter_by(id=3))
    
    output = users.mappings().all()
    
if not outputs:
    
    print("not found")

2023-03-17 14:04:13,849 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 14:04:13,851 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = %(id_1)s::INTEGER
2023-03-17 14:04:13,853 INFO sqlalchemy.engine.Engine [cached since 771.5s ago] {'id_1': 3}
2023-03-17 14:04:13,902 INFO sqlalchemy.engine.Engine COMMIT
not found


### count number of rows in table

In [22]:
# CREATE A SESSION OBJECT TO INITIATE QUERY IN DATABASE
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()
 
# SELECT COUNT(*) FROM Actor
result = session.query(User).count()

print(result)

2023-03-17 13:52:44,578 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 13:52:44,588 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account) AS anon_1
2023-03-17 13:52:44,589 INFO sqlalchemy.engine.Engine [generated in 0.00134s] {}
2
